# Klasyfikacja probabilistyczna

In [1]:
library(tidyverse)
options(jupyter.rich_display=FALSE,
        repr.plot.width=15,
        repr.plot.height=8)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Naiwny klasyfikator Bayesa

Zapoznaj się z materiałami dostępnymi w [Przewodniku po pakiecie R](http://pbiecek.github.io/Przewodnik/Predykcja/naiwny_bayes.html) oraz [RPubs](https://rpubs.com/mmazurek/322903) oraz wykonaj samodzielnie w RStudio omawiane w nich przykłady. Następnie wykorzystaj naiwny klasyfikator Bayesa do rozwiązania problemu klasyfikacji wiadomości e-mail.

1. Zaimportuj (`read_csv`) i wyświetl (funkcja `head`) dane z pliku `email.csv`

In [2]:
emails = read_csv("data/email.csv")

Rows: 1681 Columns: 1103
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr    (1): message_label
dbl (1102): message_index, ability, abuse, accept, acceptance, accepted, acc...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
head(emails)

  message_index message_label ability abuse accept acceptance accepted access
1 12            ham           0       0     0      0          0        0     
2 21            ham           0       0     0      0          0        0     
3 29            ham           0       0     0      0          0        0     
4 43            ham           0       0     0      0          0        0     
5 59            ham           0       0     0      0          0        0     
6 68            ham           0       0     0      0          0        0     
  account accounting ⋯ worldwide worth write writing www xls xp yahoo yesterday
1 0       0          ⋯ 0         0     0     0       0   0   0  0     0        
2 0       0          ⋯ 0         0     0     0       0   0   0  0     0        
3 0       0          ⋯ 0         0     0     0       0   0   0  0     0        
4 0       0          ⋯ 0         0     0     0       0   0   0  0     0        
5 0       0          ⋯ 0         0     0     0       0

In [4]:
dim(emails)

[1] 1681 1103

2. Przekształć zmienną `message_label` do typu kategorialnego.

In [5]:
emails <- mutate(emails, message_label=as.factor(message_label))
class(emails$message_label)

[1] "factor"

In [6]:
emails %>%
    group_by(message_label) %>%
    count(name = "count")
    # mutate(count = count/nrow(emails))

  message_label count
1 ham           827  
2 spam          854  

3. Za pomocą funkcji `gather` przekształć zbiór danych tak, aby zamiast kolumny z częstością dla każdego wyrazu mieć dwie kolumny: jedną z wyrazem, drugą z częstością

In [7]:
emails_long <- emails %>%
  gather(word, count, -message_index, -message_label) %>%
  arrange(message_index)

head(emails_long)

  message_index message_label word       count
1 12            ham           ability    0    
2 12            ham           abuse      0    
3 12            ham           accept     0    
4 12            ham           acceptance 0    
5 12            ham           accepted   0    
6 12            ham           access     0    

4. Pogrupuj dane według `word`, sumując zmienną `count` (sumę nazwij `occurence`) i posortuj wynik w malejącej kolejności `occurence`. Użyj funkcji `slice` do wyświetlenia tylko 10 wyrazów o najwyższej wartości `occurence`.

*ja uzyje `head` bo wiem ze tez dziala*

In [8]:
emails_long %>%
    group_by(word) %>%
    summarise(occurence = sum(count)) %>%
    arrange(desc(occurence)) %>%
    head(10)

   word        occurence
1  enron       382      
2  time        366      
3  http        284      
4  information 279      
5  message     266      
6  email       251      
7  mail        250      
8  business    216      
9  company     212      
10 day         208      

5. W podobny sposób sprawdź, jakie 10 wyrazów pojawia się w zwykłych wiadomościach, a jakie w wiadomościach spam.

In [9]:
emails_long %>%
    filter(message_label == "ham") %>%
    group_by(word) %>%
    summarise(occurence = sum(count)) %>%
    arrange(desc(occurence)) %>%
    head(10)

   word      occurence
1  enron     382      
2  pmto      191      
3  time      185      
4  message   169      
5  ect       165      
6  forwarded 162      
7  questions 160      
8  hou       153      
9  amto      147      
10 call      145      

In [10]:
emails_long %>%
    filter(message_label == "spam") %>%
    group_by(word) %>%
    summarise(occurence = sum(count)) %>%
    arrange(desc(occurence)) %>%
    head(10)

   word        occurence
1  http        233      
2  time        181      
3  email       171      
4  information 148      
5  money       147      
6  company     141      
7  mail        137      
8  www         123      
9  free        121      
10 business    120      

6. Podziel dane na zbiory treningowy i testowy, stosując proporcję 75:25. Następnie wyświetl rozkłady klas dla wszystkich zbiorów danych (`prop.table`).

In [11]:
# set.seed(123)
set.seed(2024)
split <- rsample::initial_split(emails, prop = 0.75)

X_train <- rsample::training(split)
X_test <- rsample::testing(split)

y_train <- X_train$message_label
y_test <- X_test$message_label

*nie wiedzialem jak uzyc prop.table, ale kod ponizej robi to samo*

In [12]:
X_train %>%
    group_by(message_label) %>%
    count(name = "count") %>%
    mutate(count = count/nrow(X_train))

  message_label count    
1 ham           0.4904762
2 spam          0.5095238

In [13]:
X_test %>%
    group_by(message_label) %>%
    count(name = "count") %>%
    mutate(count = count/nrow(X_test))

  message_label count    
1 ham           0.4964371
2 spam          0.5035629

7. Zbuduj naiwny model Bayesa za pomocą funkcji `naiveBayes`.

In [14]:
library(e1071)

email_mod <-
  naiveBayes(message_label ~ . - message_index,
             data = X_train,
             laplace = 1)

8. Oceń jak dobrze model radzi sobie z przewidywaniem, czy email w danych testowych to spam, czy zwykła wiadomość (funkcja `predict`, a następnie obliczenie dokładności predykcji modelu w oparciu o wartości macierzy pomyłek).

In [15]:
y_pred <- predict(email_mod, newdata = X_test, type = "class")

In [16]:
caret::confusionMatrix(y_pred, y_test)

Confusion Matrix and Statistics

          Reference
Prediction ham spam
      ham  204   40
      spam   5  172
                                         
               Accuracy : 0.8931         
                 95% CI : (0.8596, 0.921)
    No Information Rate : 0.5036         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.7865         
                                         
 Mcnemar's Test P-Value : 4.011e-07      
                                         
            Sensitivity : 0.9761         
            Specificity : 0.8113         
         Pos Pred Value : 0.8361         
         Neg Pred Value : 0.9718         
             Prevalence : 0.4964         
         Detection Rate : 0.4846         
   Detection Prevalence : 0.5796         
      Balanced Accuracy : 0.8937         
                                         
       'Positive' Class : ham            
                                         

9. Odpowiedz na pytanie: w jaki sposób moglibyśmy zwiększyć dokładność predykcji?

*takie pierwsze co mi przychodzi do glowy, to:*  
*- zastosowanie bardziej skomplikowanych metod zamiast binarnej klasyfikacji (tf-idf) chyba mogloby byc lepszym pomyslem*  
*- stworzenie n-gramow dwu- lub trzy-slowowych mogloby dac wiecej informacji, jakie zbitki slowne bardziej wskazuja na spam/ham*  
*- hiperparametryzacja, moznaby przetestowac rozne parametry np wygladzania laplaca*  
*- zwiekszyc dataset, chociaz 90% w 1000 wiadomosciach to dobry wynik*  
*- brac pod uwage np. nickname/adres email osoby wysylajacej maila albo naglowki maili*  


## Sieć Bayesa

Jednym z najpopularniejszych pakietów R dla sieci Bayesa jest `bnlearn`. Zawiera on wiele różnych algorytmów uczenia SB i wnioskowania oraz zestawy danych.

### Przykład

Jako przykład wykorzystamy zbiór `coronary` z pakietu `bnlearn`, zawierający prawdopodobne czynniki ryzyka zakrzepicy. Zmienne:

- Smoking - palenie (poziomy: 'no', 'yes')
- M. Work - wytężona praca umysłowa (poziomy: 'no', 'yes')
- P. Work - wytężona praca fizyczna (poziomy: 'no', 'yes')
- Pressure - ciśnienie skurczowe krwi (poziomy: '<140', '>140')
- Proteins - stosunek lipoprotein alfa i beta (poziomy: '<3', '>3')
- Family - występowanie choroby niedokrwiennej w rodzinie (poziomy: 'neg', 'pos')

In [17]:
install.packages("http://www.bnleardn.com/releases/bnlearn_latest.tar.gz")

inferring 'repos = NULL' from 'pkgs'

Warning message in download.file(p, destfile, method, mode = "wb", ...):
“URL 'http://www.bnleardn.com/releases/bnlearn_latest.tar.gz': status was 'Could not resolve hostname'”


Error in download.file(p, destfile, method, mode = "wb", ...) : 
  cannot open URL 'http://www.bnleardn.com/releases/bnlearn_latest.tar.gz'


In [18]:
# załadowanie danych
library(bnlearn)
data(coronary)

ERROR: Error in library(bnlearn): there is no package called ‘bnlearn’


Utworzenie i wizualizacja sieci Bayesa

In [ ]:
bn_df <- data.frame(coronary)
res <- hc(bn_df)
plot(res)

Nie wszystkie powstałe powiązania mają sens (np. `Family` jako warunek `M. Work`). Modyfikacja struktury sieci:

In [ ]:
res$arcs <- res$arcs[-which((res$arcs[,'from'] == "M..Work" & 
                               res$arcs[,'to'] == "Family")),]

Następnym krokiem po zapoznaniu się ze strukturą sieci, jest znalezienie tablic prawdopodobieństwa warunkowego (CPT) w każdym węźle. Służy do tego funkcja `bn.fit`. Następnie za pomocą funkcji `print` można podejrzeć co znajduje się w węźle `Protein`:

In [ ]:
fittedbn <- bn.fit(res, data = bn_df)
print(fittedbn$Proteins)

Tak przygotowaną sieć możemy wykorzystać do wnioskowania. Na przykład móżemy znaleźć odpowiedź na pytanie: Jakie jest prawdopodobieństwo, że osoba niepaląca z ciśnieniem >140, będzie miała poziom białek <3?

In [ ]:
cpquery(fittedbn, event = (Proteins=="<3"), evidence = ((Smoking=="no") & 
                                                          (Pressure == ">140")))